# Modelo item-based
- Métrica de avaliação: número de cliques na notícia
- Pega mais "populares" para usuários não conhecidos

In [3]:
import warnings
warnings.filterwarnings('ignore')


In [4]:
import pandas as pd

df_training = pd.read_csv("../data/training_data.csv")
df_news = pd.read_csv("../data/noticias.csv")

In [5]:
df_training = df_training.drop(columns=["Unnamed: 0"])
df_news= df_news.drop(columns=["Unnamed: 0"])

In [6]:
df_training.head()

,userId,history,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
0,0,c8aab885-433d-4e46-8066-479f40ba7fb2,76,20380,50.30,2
1,0,68d2039c-c9aa-456c-ac33-9b2e8677fba7,38,21184,18.18,1
2,0,13e423ce-1d69-4c78-bc18-e8c8f7271964,41,35438,16.46,1
3,1,3325b5a1-979a-4cb3-82b6-63905c9edbe8,7,6049,25.35,1
4,1,fe856057-f97d-419f-ab1c-97c5c3e0719c,80,210489,45.66,1


In [7]:
df_news.head()

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic..."


In [8]:
#considerando apenas noticias com mais de 30 cliques
clicks_counts = pd.DataFrame(df_training["history"].value_counts())
rare_news = clicks_counts[df_training["history"].value_counts() <= 30].index
common_news = df_training[~df_training["history"].isin(rare_news)]

In [9]:
df_training.shape

(1426291, 6)

In [10]:
common_news.shape

(1085862, 6)

In [11]:
common_news.head(130)

,userId,history,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
0,0,c8aab885-433d-4e46-8066-479f40ba7fb2,76,20380,50.30,2
1,0,68d2039c-c9aa-456c-ac33-9b2e8677fba7,38,21184,18.18,1
2,0,13e423ce-1d69-4c78-bc18-e8c8f7271964,41,35438,16.46,1
4,1,fe856057-f97d-419f-ab1c-97c5c3e0719c,80,210489,45.66,1
5,1,d1da8d3f-bf30-4776-8d75-5f4e24cc134e,2,8672,35.30,1
...,...,...,...,...,...,...
184,3,39410577-0d94-4790-ac41-81362bc8c915,1,110000,19.11,1
185,3,5db25cd8-32e2-4502-9208-b62d0e13b015,1,90000,17.84,1
186,3,14a54c8d-3155-4045-83bc-ae01af95a034,1,190000,16.56,1
187,3,bcb7755a-5046-4bd7-81cd-8ead61a27ba7,3,140000,14.89,1


In [12]:
# Combine two datasets

df_merged = pd.merge(common_news,df_news, left_on='history', right_on='page', how='left')
df_merged = df_merged.drop(columns=['page'])
df_merged.head(130)

,userId,history,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,url,issued,modified,title,body,caption
0,0,c8aab885-433d-4e46-8066-479f40ba7fb2,76,20380,50.30,2,http://g1.globo.com/sc/santa-catarina/noticia/...,2022-03-19 21:03:21+00:00,2022-03-19 21:03:21+00:00,"Você viu? 'Musa das Estradas' faz vídeo de pé,...",Caminhoneira Aline Füchter em pé em casa\nRepr...,Caminhoneira Aline Füchter ficou em pé em fren...
1,0,68d2039c-c9aa-456c-ac33-9b2e8677fba7,38,21184,18.18,1,http://g1.globo.com/rj/rio-de-janeiro/noticia/...,2021-11-01 03:01:20+00:00,2021-11-01 13:20:44+00:00,'Mulher-Gato' foi proibida de entrar na Maré a...,"Polícia Civil do Rio prende Mulher-Gato, apont...","Luana Rabello, segundo a polícia, é muito famo..."
2,0,13e423ce-1d69-4c78-bc18-e8c8f7271964,41,35438,16.46,1,http://g1.globo.com/sc/santa-catarina/noticia/...,2022-02-01 18:33:21+00:00,2022-02-04 20:23:50+00:00,Caminhoneira 'Musa das Estradas' mostra rosto ...,Caminhoneira 'Musa das Estradas' mostra rosto ...,"Aline Füchter chegou a Tubarão, onde mora, no ..."
3,1,fe856057-f97d-419f-ab1c-97c5c3e0719c,80,210489,45.66,1,http://g1.globo.com/sp/itapetininga-regiao/not...,2022-08-14 11:39:11+00:00,2022-08-15 15:18:15+00:00,Designer de sobrancelhas viraliza na web ao fa...,Designer de sobrancelhas viraliza na web ao fa...,"Vídeo publicado por Geizielle Ferreira Mendes,..."
4,1,d1da8d3f-bf30-4776-8d75-5f4e24cc134e,2,8672,35.30,1,http://g1.globo.com/sp/itapetininga-regiao/not...,2022-08-13 15:26:35+00:00,2022-08-13 15:26:38+00:00,Lancha se desprende de carro e atinge furgão e...,Lancha se desprende de carro e atinge furgão e...,Batida aconteceu na Rodovia João Melão (SP-255...
...,...,...,...,...,...,...,...,...,...,...,...,...
125,3,39410577-0d94-4790-ac41-81362bc8c915,1,110000,19.11,1,http://g1.globo.com/df/distrito-federal/notici...,2022-08-05 10:52:35+00:00,2022-08-06 13:23:32+00:00,Empresário atropela e mata motociclista e deix...,Empresário atropela e mata motociclista e deix...,Uma das motos foi partida ao meio; homem de 52...
126,3,5db25cd8-32e2-4502-9208-b62d0e13b015,1,90000,17.84,1,http://g1.globo.com/sp/sao-paulo/noticia/2022/...,2022-08-05 11:26:46+00:00,2022-08-05 13:37:43+00:00,"Integrante do 'Sexteto', Derico chora e lament...","Derico Sciotti, saxofonista na banda do Progra...",Derico Sciotti trabalhou com Jô Soares durante...
127,3,14a54c8d-3155-4045-83bc-ae01af95a034,1,190000,16.56,1,http://g1.globo.com/sp/santos-regiao/noticia/2...,2022-08-04 08:04:59+00:00,2022-08-04 17:39:48+00:00,Bandidos escapam pelo telhado com dinheiro apó...,Assaltantes de banco são capturados após perse...,Criminosos foram perseguidos pelos policiais m...
128,3,bcb7755a-5046-4bd7-81cd-8ead61a27ba7,3,140000,14.89,1,http://g1.globo.com/tecnologia/noticia/2022/08...,2022-08-04 03:01:35+00:00,2022-08-04 17:24:13+00:00,Faixa principal do 5G é ativada em São Paulo e...,Sinal do 5G foi liberado nesta sexta-feira (29...,"Começa a ser ofertado o 5G Standalone ou SA, q..."


In [13]:
df_merged.shape

(1085862, 12)

In [14]:
common_news["history"].value_counts().head(100000)

history
d2593c3d-2347-40d9-948c-b6065e8459a9    4282
f6b5d170-48b9-4f8e-88d4-c84b6668f3bd    3873
1f32787b-de2b-49be-8c20-ddaeae34cc22    3395
f0a78e58-ec7e-494c-9462-fbd6446a9a89    3192
6a83890a-d9e9-4f6b-a6c6-90d031785bbf    3121
                                        ... 
4da21b81-a5bd-4eb7-8c79-e168d779c523      31
aa5f3369-168f-45e7-b005-3b9a3bfcf62f      31
879a2b89-1bbb-4799-a422-77f1959cacf8      31
3a5b4ef3-e295-409c-baca-e4f06e7f3a54      31
383caa1c-2040-4437-8cfd-0ea8d04c489f      31
Name: count, Length: 5077, dtype: int64

In [15]:
matriz_usuarios_noticias = common_news.pivot(index='userId', columns='history', values='numberOfClicksHistory')

In [16]:
matriz_usuarios_noticias.head()

history,000e18cd-4bd3-4629-b6fa-cc7c2173acf9,0010fb40-55c2-480e-bd42-6cce10bdf90a,001b9bec-b1e2-4df2-9c43-8135627ce4c1,003f8e2c-a9a7-4ef9-8bee-51011e11a619,005063cf-ad1c-4a4d-bc96-fb6b4298ec3e,005a743a-0a81-4dd4-ba1f-6dc63cc33a34,00630ea8-4698-4748-9ef6-7c99f545a916,008d7a48-0864-4202-9a4e-b5d4f3c08e8b,008d8f56-7105-426d-bb81-7aab44b3aee6,00a31551-a3c4-4586-a5be-4766486583ed,...,ff6e7987-c84d-4147-8de1-e8acf8a93df7,ff7421ee-fc6f-4c5d-846a-1143fae40832,ff872490-bef6-4c14-b5ec-66ac9d24ad19,ffabc550-b315-4070-94b6-713712da8ec2,ffb8b407-8dbb-40d8-8ea1-d5e9aa14a690,ffcba96d-e286-43f0-b3cf-f2c05b0bea64,ffdb1505-f686-4ba6-a0a9-89d7246722f0,ffe56be2-f822-44ac-b7c2-6bef6e8833a6,ffeea3a6-d969-4a9a-baf0-101da68da3f9,fff8f4aa-531e-4858-9108-31ec3c0f6ee6
userId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
matriz_usuarios_noticias.loc[2,'0b5caeed-ff6f-4b9d-8375-9ad25827a4ae']

np.float64(1.0)

In [83]:
matriz_usuarios_noticias.loc[3, :].sample(10)

history
36bbc315-f866-4b25-bdd0-daa8724a6037   NaN
ebe929f9-cb63-45cf-9b75-969a7fbf9ed3   NaN
f3a1a5ac-ef7b-43b0-a079-9212460e90b5   NaN
a9d44e59-4fba-44c5-9d52-c9733e81a3ec   NaN
d1706155-a428-4cbc-88d8-c1d59d998c74   NaN
6d6bb0f0-0586-40ee-8db7-f7054e370707   NaN
ab22e5cd-715a-46f0-99d8-613b981f202c   NaN
b894ddc9-38f5-4d48-82e5-294c23eba97c   NaN
fe9e9751-34c2-4eca-8ab0-7b66b4176a95   NaN
a80614bb-c02c-4837-96ff-cb9239189dcd   NaN
Name: 3, dtype: float64

In [17]:
def recommend(random_user,df_news,df_merged,matriz_usuarios_noticias,n_recomm=5):

    random_user_new = df_merged[(df_merged["userId"] == random_user) & (df_merged["numberOfClicksHistory"] >= 1)].sort_values(by="modified", ascending=False).reset_index()

    first_new = random_user_new["history"][0]

    new_item = matriz_usuarios_noticias[first_new]
    prediction = matriz_usuarios_noticias.corrwith(new_item).sort_values(ascending=False)
    prediction = prediction.to_frame()
    recommended = prediction.head(n_recomm)
    recommended = pd.merge(recommended,df_news, left_on='history', right_on='page', how='left')
    recommended_formated = recommended.drop(columns=[0,'page'])
    
    return recommended_formated.head(n_recomm)

In [33]:
random_user = int(pd.Series(df_training.userId).sample(1, random_state=45).values)
N_RECOMMENDATIONS = 5

try:
    #faz recomendação baseado em similaridade com outros usuarios
    recommends = recommend(211, df_news, df_merged, matriz_usuarios_noticias, n_recomm=N_RECOMMENDATIONS)
except:
    #recomenda mais populares caso usuário não exista
    recommends = common_news["history"].value_counts().head(5)
    recommends = pd.merge(recommends,df_news, left_on='history', right_on='page', how='left')
    recommends = recommends.drop(columns=['page']) 

recommends

,url,issued,modified,title,body,caption
0,http://g1.globo.com/saude/noticia/2022/07/01/o...,2022-07-01 19:46:13+00:00,2022-07-01 20:04:02+00:00,OMS pede ação urgente contra varíola dos macac...,Varíola dos macacos é semelhante à varíola que...,"Em apenas duas semanas, casos da doença tripli..."
1,http://g1.globo.com/mundo/noticia/2022/07/24/z...,2022-07-25 01:05:07+00:00,2022-07-25 01:34:07+00:00,Zelensky critica neutralidade brasileira: 'Se ...,Zelensky fala sobre relação com Putin: 'Nem pr...,Presidente da Ucrânia deu uma entrevista à Glo...
2,http://g1.globo.com/ce/ceara/noticia/2022/07/2...,2022-07-26 09:35:10+00:00,2022-07-26 09:57:15+00:00,Adolescente é morto a tiros pela PM quando saí...,Jovem é morto em abordagem policial quando saí...,Paulo Vitor Oliveira estava em uma esquina con...
3,http://g1.globo.com/mg/triangulo-mineiro/notic...,2022-07-13 11:54:52+00:00,2022-07-14 00:05:31+00:00,Mulher é morta pelo marido em MG; vídeo mostra...,Suspeito de feminicídio foge após o crime em U...,Wellen Kássia Cardoso de Melo foi morta com 5 ...
4,http://g1.globo.com/mundo/noticia/2022/07/09/o...,2022-07-09 14:42:39+00:00,2022-07-09 14:42:40+00:00,O delirante e fascinante mundo dos micropaíses,Esta é a sede do governo de Atlantium\nDivulga...,Uma micronação não é um país real por definiçã...


### Colocando modelo na classe

In [30]:
class CustomModel():

    def __init__(self,random_user,df_news,df_merged,matriz_usuarios_noticias):
        self.random_user = random_user
        self.df_news = df_news
        self.df_merged = df_merged
        self.matriz_usuarios_noticias = matriz_usuarios_noticias

    def recommend(self,n_recomm=5):

        random_user_new = self.df_merged[(self.df_merged["userId"] == self.random_user) & (self.df_merged["numberOfClicksHistory"] >= 1)].sort_values(by="modified", ascending=False).reset_index()

        first_new = random_user_new["history"][0]

        new_item = self.matriz_usuarios_noticias[first_new]
        prediction = self.matriz_usuarios_noticias.corrwith(new_item).sort_values(ascending=False)
        prediction = prediction.to_frame()
        recommended = prediction.head(n_recomm)
        recommended = pd.merge(recommended,self.df_news, left_on='history', right_on='page', how='left')
        recommended_formated = recommended.drop(columns=[0,'page'])

        return recommended_formated.head(n_recomm)
    
    def predict(self):
        try:
            #faz recomendação baseado em similaridade com outros usuarios
            recommends = self.recommend()
        except:
            #recomenda mais populares caso usuário não exista
            recommends = common_news["history"].value_counts().head(5)
            recommends = pd.merge(recommends,df_news, left_on='history', right_on='page', how='left')
            recommends = recommends.drop(columns=['page']) 

            return recommends

        return recommends

In [34]:
user_id = 211
model = CustomModel(user_id, df_news, df_merged, matriz_usuarios_noticias)
result = model.predict()
result

,url,issued,modified,title,body,caption
0,http://g1.globo.com/saude/noticia/2022/07/01/o...,2022-07-01 19:46:13+00:00,2022-07-01 20:04:02+00:00,OMS pede ação urgente contra varíola dos macac...,Varíola dos macacos é semelhante à varíola que...,"Em apenas duas semanas, casos da doença tripli..."
1,http://g1.globo.com/mundo/noticia/2022/07/24/z...,2022-07-25 01:05:07+00:00,2022-07-25 01:34:07+00:00,Zelensky critica neutralidade brasileira: 'Se ...,Zelensky fala sobre relação com Putin: 'Nem pr...,Presidente da Ucrânia deu uma entrevista à Glo...
2,http://g1.globo.com/ce/ceara/noticia/2022/07/2...,2022-07-26 09:35:10+00:00,2022-07-26 09:57:15+00:00,Adolescente é morto a tiros pela PM quando saí...,Jovem é morto em abordagem policial quando saí...,Paulo Vitor Oliveira estava em uma esquina con...
3,http://g1.globo.com/mg/triangulo-mineiro/notic...,2022-07-13 11:54:52+00:00,2022-07-14 00:05:31+00:00,Mulher é morta pelo marido em MG; vídeo mostra...,Suspeito de feminicídio foge após o crime em U...,Wellen Kássia Cardoso de Melo foi morta com 5 ...
4,http://g1.globo.com/mundo/noticia/2022/07/09/o...,2022-07-09 14:42:39+00:00,2022-07-09 14:42:40+00:00,O delirante e fascinante mundo dos micropaíses,Esta é a sede do governo de Atlantium\nDivulga...,Uma micronação não é um país real por definiçã...
